In [1]:
# !pip install elasticsearch

In [2]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan
import warnings
# warnings.filterwarnings('ignore')

import pandas as pd
import matplotlib.pyplot as plt

import pandas as pd
import re
from datetime import datetime

from dataQuery import SendQuery

### Import Read_{date}.txt

In [3]:
IHM_date = '0922'
IHM_path = './00007_btl_new/demo_0922/Read_2023' + IHM_date + '.txt'

with open(IHM_path, 'r') as file:
    IHM_data = file.read()

# matches = re.findall(r'^(.*EtatVoyant\s*\|\s*(?:2|4).*)$', IHM_data, re.MULTILINE)
matches = re.findall(r'^.*EtatVoyant.*$', IHM_data, re.MULTILINE)

voyant_list = []
for match in matches:
    voyant_list.append(match.replace('\x00', ''))

In [4]:
voyant_list

['2023-09-22 11:11:02.468 +02:00 | ["Read"] | EtatVoyant | 2',
 '2023-09-22 11:13:44.779 +02:00 | ["Read"] | EtatVoyant | 1',
 '2023-09-22 11:15:46.511 +02:00 | ["Read"] | EtatVoyant | 2',
 '2023-09-22 11:17:38.644 +02:00 | ["Read"] | EtatVoyant | 1',
 '2023-09-22 11:20:04.305 +02:00 | ["Read"] | EtatVoyant | 2',
 '2023-09-22 11:21:13.502 +02:00 | ["Read"] | EtatVoyant | 2',
 '2023-09-22 11:21:20.105 +02:00 | ["Read"] | EtatVoyant | 1',
 '2023-09-22 11:21:21.536 +02:00 | ["Read"] | EtatVoyant | 2',
 '2023-09-22 11:21:57.286 +02:00 | ["Read"] | EtatVoyant | 1',
 '2023-09-22 11:32:40.773 +02:00 | ["Read"] | EtatVoyant | 1',
 '2023-09-22 11:32:48.363 +02:00 | ["Read"] | EtatVoyant | 2',
 '2023-09-22 11:35:45.261 +02:00 | ["Read"] | EtatVoyant | 2',
 '2023-09-22 11:39:10.171 +02:00 | ["Read"] | EtatVoyant | 2',
 '2023-09-22 11:40:05.043 +02:00 | ["Read"] | EtatVoyant | 1',
 '2023-09-22 11:42:46.333 +02:00 | ["Read"] | EtatVoyant | 2',
 '2023-09-22 11:46:45.300 +02:00 | ["Read"] | EtatVoyan

In [5]:
pattern = r'(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}\.\d{3}) \+\d{2}:\d{2} \| \["Read"\] \| EtatVoyant \| (\d)'
matches = [re.search(pattern, data) for data in voyant_list]
dates = [match.group(1) for match in matches if match]
numbers = [match.group(2) for match in matches if match]

df_voyant_0922 = pd.DataFrame({
    'Date': dates,
    'EtatVoyant': numbers
})
df_voyant_0922['Date'] = pd.to_datetime(df_voyant_0922['Date'], format='%Y-%m-%d %H:%M:%S.%f')

df_voyant_0922

,Date,EtatVoyant
0,2023-09-22 11:11:02.468,2
1,2023-09-22 11:13:44.779,1
2,2023-09-22 11:15:46.511,2
3,2023-09-22 11:17:38.644,1
4,2023-09-22 11:20:04.305,2
...,...,...
745,2023-09-22 19:57:35.643,1
746,2023-09-22 19:58:38.061,2
747,2023-09-22 19:58:39.561,1
748,2023-09-22 19:58:51.900,2


### Détection de double bouteilles

In [6]:
import re
from datetime import datetime
import numpy as np

In [7]:
def find_etat_voyant(df, ini_tmp, in_tmp_next):
#     print(ini_tmp, in_tmp_next)
    previous_idx = df[df['Date'] <= ini_tmp].last_valid_index()
    next_idx     = df[df['Date'] <= in_tmp_next].last_valid_index()
#     next_idx     = int(df[df['Date'] >= in_tmp_next].index.values[0])
#     print(previous_idx, next_idx)
    if previous_idx is not None and next_idx is not None and previous_idx <= next_idx:
        etat_values = df.loc[previous_idx:next_idx, 'EtatVoyant'].astype(int).values
#         print(etat_values)
#         etat_product = np.prod(etat_values)
        return etat_values
    else:
        return [2]

In [30]:
def find_close_attempts(log_file, df_voyant):
    with open(log_file, 'r') as f:
        lines = f.readlines()
    print("log file has %d lines" % len(lines))

    pattern     = r"\s*(.*)INFO - start  at\s*(.*)"
    pattern_ini = r"\s*(.*)INFO - obj stats - img_ini\s*(.*)"
    pattern_tmp = r"\s*(.*)INFO - image  at\s*(.*)"
    pattern_pos = r"\s*(.*)INFO - obj is at\s*(.*)"
    pattern_in  = r"\s*(.*)INFO - obj stats - img_in\s*(.*)"
    # pattern = r"\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2},\d{3} - INFO - video is saved as\s*(.*)"

    i = 0
    while i <= len(lines) - 1:
        img_ini = re.search(pattern_ini, lines[i])
        i += 1
        if not img_ini:
            continue
            
        ini_tmp = re.search(pattern_tmp, lines[i])
        ini_tmp = datetime.strptime(str(ini_tmp[0][45:]), "%Y-%m-%d %H:%M:%S.%f")
        
        ini_pos = re.search(pattern_pos, lines[i + 1])
        ini_pos = str(ini_pos[0][45:])
        ini_pos = np.array([float(num) for num in ini_pos.strip("[]").split()])

        i += 7
        img_in = re.search(pattern_in, lines[i])
        if img_in:
            in_exist = True
            i += 1
            in_tmp = re.search(pattern_tmp, lines[i])
            in_tmp = datetime.strptime(str(in_tmp[0][45:]), "%Y-%m-%d %H:%M:%S.%f")
            # print("in ", in_tmp)
            continue
        else:
            in_exist = False
            # print("in  rien")
        
        if min(ini_pos[0], ini_pos[2]) <= 240:
        # if max(ini_pos[0], ini_pos[2]) >= 420:
            ini_pos = 'out'
        else:
            ini_pos = 'in'

        j = i
        while j <= len(lines) - 2:
            j += 1
            img_ini_next = re.search(pattern, lines[j])
            if not img_ini_next:
                continue
            # print(img_ini_next[0])
            video = str(lines[j-2])
            j += 8
            ini_tmp_next = re.search(pattern_tmp, lines[j])
            ini_tmp_next = datetime.strptime(str(ini_tmp_next[0][45:]), "%Y-%m-%d %H:%M:%S.%f")
            # print("ini next", ini_tmp_next)

            j += 7
            img_in_next = re.search(pattern_in, lines[j])
            if img_in_next:
                in_exist_next = True
                j += 1
                in_tmp_next = re.search(pattern_tmp, lines[j])
                in_tmp_next = datetime.strptime(str(in_tmp_next[0][45:]), "%Y-%m-%d %H:%M:%S.%f")
                # print("in  next", in_tmp_next)
                
            else:
                in_exist_next = False
                # print("in  next rien")
            break
        
        if ini_pos == 'out' and not in_exist and in_exist_next and 0.3 <= abs(ini_tmp_next - ini_tmp).total_seconds() <= 3 and 4 not in find_etat_voyant(df_voyant, ini_tmp, in_tmp_next):
            print(video)
            print("diff tmp", abs(ini_tmp_next - ini_tmp))
            print("ini", ini_tmp)
#             print(find_etat_voyant(df_voyant, ini_tmp))
            print(ini_pos)
            print("in  rien")

            print("ini next", ini_tmp_next)
#             print(find_etat_voyant(df_voyant, ini_tmp_next))
            
            print("in  next", in_tmp_next)
#             print(find_etat_voyant(df_voyant, in_tmp_next))
            print(find_etat_voyant(df_voyant, ini_tmp, in_tmp_next))
            
        if j == len(lines) - 1:
            break

    return 0

### 0922

In [10]:
log_file = './00007_btl_new/demo_0922/run3/log.txt'
# log_file = '../../ia_ob2_btl_coupee/run2_dcp/log.txt'

attempts = find_close_attempts(log_file, df_voyant_0922)

log file has 33050 lines
2023-09-22 16:15:36,834 - INFO - video is saved as ./demo/run3/20230922161535304_btl_126
diff tmp 0:00:01.572869
ini 2023-09-22 16:15:35.424867
in  rien
ini next 2023-09-22 16:15:36.997736
in  next 2023-09-22 16:15:37.194220
2023-09-22 16:21:15,547 - INFO - video is saved as ./demo/run3/20230922162114839_btl_181
diff tmp 0:00:00.817355
ini 2023-09-22 16:21:14.977176
in  rien
ini next 2023-09-22 16:21:15.794531
in  next 2023-09-22 16:21:16.326190
2023-09-22 16:26:44,574 - INFO - video is saved as ./demo/run3/20230922162643962_btl_306
diff tmp 0:00:02.677807
ini 2023-09-22 16:26:44.089032
in  rien
ini next 2023-09-22 16:26:46.766839
in  next 2023-09-22 16:26:46.905333
2023-09-22 16:38:34,451 - INFO - video is saved as ./demo/run3/20230922163832202_btl_583
diff tmp 0:00:01.841013
ini 2023-09-22 16:38:34.169773
in  rien
ini next 2023-09-22 16:38:32.328760
in  next 2023-09-22 16:38:32.468907
2023-09-22 16:39:47,011 - INFO - video is saved as ./demo/run3/202309221639

### 0923

In [11]:
IHM_date = '0923'
IHM_path = './00007_btl_new/demo_0922/Read_2023' + IHM_date + '.txt'

with open(IHM_path, 'r') as file:
    IHM_data = file.read()

# matches = re.findall(r'^(.*EtatVoyant\s*\|\s*(?:2|4).*)$', IHM_data, re.MULTILINE)
matches = re.findall(r'^.*EtatVoyant.*$', IHM_data, re.MULTILINE)

voyant_list = []
for match in matches:
    voyant_list.append(match.replace('\x00', ''))

pattern = r'(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}\.\d{3}) \+\d{2}:\d{2} \| \["Read"\] \| EtatVoyant \| (\d)'
matches = [re.search(pattern, data) for data in voyant_list]
dates = [match.group(1) for match in matches if match]
numbers = [match.group(2) for match in matches if match]

df_voyant_0923 = pd.DataFrame({
    'Date': dates,
    'EtatVoyant': numbers
})
df_voyant_0923['Date'] = pd.to_datetime(df_voyant_0923['Date'], format='%Y-%m-%d %H:%M:%S.%f')

In [31]:
log_file = './00007_btl_new/demo_0922/run4/log.txt'
attempts = find_close_attempts(log_file, df_voyant_0923)

log file has 8140 lines
2023-09-23 08:31:35,044 - INFO - video is saved as ./demo/run4/20230923083132602_btl_208

diff tmp 0:00:02.672051
ini 2023-09-23 08:31:30.054685
out
in  rien
ini next 2023-09-23 08:31:32.726736
in  next 2023-09-23 08:31:34.156438
[1]
2023-09-23 08:32:37,077 - INFO - video is saved as ./demo/run4/20230923083235536_btl_224

diff tmp 0:00:01.787813
ini 2023-09-23 08:32:33.864351
out
in  rien
ini next 2023-09-23 08:32:35.652164
in  next 2023-09-23 08:32:35.789554
[1 2]
2023-09-23 08:34:23,846 - INFO - video is saved as ./demo/run4/20230923083422877_btl_231

diff tmp 0:00:00.879437
ini 2023-09-23 08:34:22.116196
out
in  rien
ini next 2023-09-23 08:34:22.995633
in  next 2023-09-23 08:34:23.178294
[1]


In [94]:
log_file = './00007_btl_new/demo_0922/run5/log.txt'
attempts = find_close_attempts(log_file, df_voyant_0923)

log file has 3341 lines


In [95]:
log_file = './00007_btl_new/demo_0922/run6/log.txt'
attempts = find_close_attempts(log_file, df_voyant_0923)

log file has 6 lines


In [96]:
log_file = './00007_btl_new/demo_0922/run7/log.txt'
attempts = find_close_attempts(log_file, df_voyant_0923)

log file has 7061 lines
2023-09-23 09:39:01,480 - INFO - video is saved as ./demo/run7/20230923093859164_btl_47
diff tmp 0:00:02.892199
ini 2023-09-23 09:38:59.291386
in  rien
ini next 2023-09-23 09:39:02.183585
in  next 2023-09-23 09:39:02.531235
2023-09-23 09:56:11,293 - INFO - video is saved as ./demo/run7/20230923095610620_btl_308
diff tmp 0:00:00.887015
ini 2023-09-23 09:56:10.749179
in  rien
ini next 2023-09-23 09:56:11.636194
in  next 2023-09-23 09:56:11.937722
2023-09-23 10:01:08,454 - INFO - video is saved as ./demo/run7/20230923100107736_btl_379
diff tmp 0:00:01.845034
ini 2023-09-23 10:01:07.861011
in  rien
ini next 2023-09-23 10:01:09.706045
in  next 2023-09-23 10:01:09.843432


In [97]:
log_file = './00007_btl_new/demo_0922/run8/log.txt'
attempts = find_close_attempts(log_file, df_voyant_0923)

log file has 281 lines


In [98]:
log_file = './00007_btl_new/demo_0922/run9/log.txt'
attempts = find_close_attempts(log_file, df_voyant_0923)

log file has 8968 lines
2023-09-23 11:09:19,614 - INFO - video is saved as ./demo/run9/20230923110919278_btl_67
diff tmp 0:00:00.651720
ini 2023-09-23 11:09:19.125371
in  rien
ini next 2023-09-23 11:09:19.777091
in  next 2023-09-23 11:09:20.078480


In [99]:
log_file = './00007_btl_new/demo_0922/run10/log.txt'
attempts = find_close_attempts(log_file, df_voyant_0923)

log file has 13 lines


In [100]:
log_file = './00007_btl_new/demo_0922/run11/log.txt'
attempts = find_close_attempts(log_file, df_voyant_0923)
# comportement de triche

log file has 4949 lines
2023-09-23 11:50:41,315 - INFO - video is saved as ./demo/run11/20230923115040712_btl_38
diff tmp 0:00:00.893983
ini 2023-09-23 11:50:40.835987
in  rien
ini next 2023-09-23 11:50:41.729970
in  next 2023-09-23 11:50:41.984342
2023-09-23 11:51:34,533 - INFO - video is saved as ./demo/run11/20230923115132116_btl_45
diff tmp 0:00:02.543814
ini 2023-09-23 11:51:32.249498
in  rien
ini next 2023-09-23 11:51:34.793312
in  next 2023-09-23 11:51:35.098481
2023-09-23 11:51:41,982 - INFO - video is saved as ./demo/run11/20230923115141297_btl_48
diff tmp 0:00:01.200477
ini 2023-09-23 11:51:41.423496
in  rien
ini next 2023-09-23 11:51:42.623973
in  next 2023-09-23 11:51:43.053906
2023-09-23 11:52:12,348 - INFO - video is saved as ./demo/run11/20230923115211685_btl_53
diff tmp 0:00:00.725865
ini 2023-09-23 11:52:11.809479
in  rien
ini next 2023-09-23 11:52:12.535344
in  next 2023-09-23 11:52:12.736245
2023-09-23 11:52:15,920 - INFO - video is saved as ./demo/run11/202309231152

In [101]:
log_file = './00007_btl_new/demo_0922/run12/log.txt'
attempts = find_close_attempts(log_file, df_voyant_0923)

log file has 3457 lines
2023-09-23 12:13:36,486 - INFO - video is saved as ./demo/run12/20230923121334521_btl_15
diff tmp 0:00:01.992142
ini 2023-09-23 12:13:34.650789
in  rien
ini next 2023-09-23 12:13:36.642931
in  next 2023-09-23 12:13:36.893697
2023-09-23 12:20:53,177 - INFO - video is saved as ./demo/run12/20230923122052592_btl_104
diff tmp 0:00:00.982623
ini 2023-09-23 12:20:52.714997
in  rien
ini next 2023-09-23 12:20:53.697620
in  next 2023-09-23 12:20:54.053272
2023-09-23 12:22:52,663 - INFO - video is saved as ./demo/run12/20230923122251176_btl_147
diff tmp 0:00:01.943286
ini 2023-09-23 12:22:51.300468
in  rien
ini next 2023-09-23 12:22:53.243754
in  next 2023-09-23 12:22:53.707299
2023-09-23 12:25:49,562 - INFO - video is saved as ./demo/run12/20230923122548869_btl_185
diff tmp 0:00:02.995531
ini 2023-09-23 12:25:48.992113
in  rien
ini next 2023-09-23 12:25:51.987644
in  next 2023-09-23 12:25:52.191772


In [102]:
log_file = './00007_btl_new/demo_0922/run13/log.txt'
attempts = find_close_attempts(log_file, df_voyant_0923)

log file has 1283 lines
2023-09-23 12:44:16,941 - INFO - video is saved as ./demo/run13/20230923124416345_btl_73
diff tmp 0:00:00.672521
ini 2023-09-23 12:44:16.472945
in  rien
ini next 2023-09-23 12:44:17.145466
in  next 2023-09-23 12:44:17.821768


In [103]:
log_file = './00007_btl_new/demo_0922/run14/log.txt'
attempts = find_close_attempts(log_file, df_voyant_0923)

log file has 555 lines
2023-09-23 13:08:05,209 - INFO - video is saved as ./demo/run14/20230923130804540_btl_4
diff tmp 0:00:00.705990
ini 2023-09-23 13:08:04.664340
in  rien
ini next 2023-09-23 13:08:05.370330
in  next 2023-09-23 13:08:05.818000
2023-09-23 13:08:39,250 - INFO - video is saved as ./demo/run14/20230923130836672_btl_14
diff tmp 0:00:02.901721
ini 2023-09-23 13:08:36.794979
in  rien
ini next 2023-09-23 13:08:39.696700
in  next 2023-09-23 13:08:40.056438


In [104]:
log_file = './00007_btl_new/demo_0922/run15/log.txt'
attempts = find_close_attempts(log_file, df_voyant_0923)

log file has 623 lines


In [105]:
log_file = './00007_btl_new/demo_0922/run16/log.txt'
attempts = find_close_attempts(log_file, df_voyant_0923)

log file has 6096 lines
2023-09-23 14:10:34,231 - INFO - video is saved as ./demo/run16/20230923141030345_btl_343
diff tmp 0:00:01.465636
ini 2023-09-23 14:10:31.933457
in  rien
ini next 2023-09-23 14:10:30.467821
in  next 2023-09-23 14:10:31.476122
2023-09-23 14:12:10,587 - INFO - video is saved as ./demo/run16/20230923141209737_btl_376
diff tmp 0:00:01.030655
ini 2023-09-23 14:12:09.870502
in  rien
ini next 2023-09-23 14:12:10.901157
in  next 2023-09-23 14:12:11.807233


In [16]:
log_file = './00007_btl_new/demo_0922/run17/log.txt'
attempts = find_close_attempts(log_file, df_voyant_0923)

log file has 1230 lines
2023-09-23 14:22:21,465 - INFO - video is saved as ./demo/run17/20230923142220471_btl_35
diff tmp 0:00:01.180694
ini 2023-09-23 14:22:20.595038
in  rien
ini next 2023-09-23 14:22:21.775732
in  next 2023-09-23 14:22:22.026816


In [107]:
log_file = './00007_btl_new/demo_0922/run18/log.txt'
attempts = find_close_attempts(log_file, df_voyant_0923)

log file has 29 lines


In [20]:
log_file = './00007_btl_new/demo_0922/run19/log.txt'
attempts = find_close_attempts(log_file, df_voyant_0923)

log file has 37873 lines
2023-09-23 14:43:52,199 - INFO - video is saved as ./demo/run19/20230923144351500_btl_9
diff tmp 0:00:01.319547
ini 2023-09-23 14:43:51.620841
out
in  rien
ini next 2023-09-23 14:43:52.940388
in  next 2023-09-23 14:43:53.243771
[1]
2023-09-23 14:44:59,323 - INFO - video is saved as ./demo/run19/20230923144457470_btl_25
diff tmp 0:00:02.683750
ini 2023-09-23 14:44:57.597622
out
in  rien
ini next 2023-09-23 14:45:00.281372
in  next 2023-09-23 14:45:00.635475
[1]
2023-09-23 14:45:53,779 - INFO - video is saved as ./demo/run19/20230923144551213_btl_74
diff tmp 0:00:02.750635
ini 2023-09-23 14:45:51.338421
out
in  rien
ini next 2023-09-23 14:45:54.089056
in  next 2023-09-23 14:45:54.334422
[1]
2023-09-23 14:49:07,431 - INFO - video is saved as ./demo/run19/20230923144906615_btl_125
diff tmp 0:00:02.917253
ini 2023-09-23 14:49:06.744186
out
in  rien
ini next 2023-09-23 14:49:09.661439
in  next 2023-09-23 14:49:10.018039
[1]
2023-09-23 14:52:03,553 - INFO - video is s

2023-09-23 18:17:28,911 - INFO - video is saved as ./demo/run19/20230923181728332_btl_2203
diff tmp 0:00:01.409691
ini 2023-09-23 18:17:28.454304
out
in  rien
ini next 2023-09-23 18:17:29.863995
in  next 2023-09-23 18:17:30.194888
[1]
2023-09-23 18:26:24,046 - INFO - video is saved as ./demo/run19/20230923182623471_btl_2237
diff tmp 0:00:01.791729
ini 2023-09-23 18:26:23.593807
out
in  rien
ini next 2023-09-23 18:26:25.385536
in  next 2023-09-23 18:26:25.645108
[2 1]
2023-09-23 19:13:45,040 - INFO - video is saved as ./demo/run19/20230923191344392_btl_2275
diff tmp 0:00:00.754609
ini 2023-09-23 19:13:44.515233
out
in  rien
ini next 2023-09-23 19:13:45.269842
in  next 2023-09-23 19:13:50.383536
[1 2]


In [109]:
log_file = './00007_btl_new/demo_0922/run20/log.txt'
attempts = find_close_attempts(log_file, df_voyant_0923)

log file has 5465 lines


In [110]:
log_file = './00007_btl_new/demo_0922/run21/log.txt'
attempts = find_close_attempts(log_file, df_voyant_0923)

log file has 11924 lines
2023-09-24 09:25:40,865 - INFO - video is saved as ./demo/run21/20230924092540206_btl_119
diff tmp 0:00:00.881836
ini 2023-09-24 09:25:40.326954
in  rien
ini next 2023-09-24 09:25:41.208790
in  next 2023-09-24 09:25:41.929096
2023-09-24 09:25:48,616 - INFO - video is saved as ./demo/run21/20230924092548369_btl_123
diff tmp 0:00:00.614386
ini 2023-09-24 09:25:48.158658
in  rien
ini next 2023-09-24 09:25:48.773044
in  next 2023-09-24 09:25:49.188931
2023-09-24 09:43:30,407 - INFO - video is saved as ./demo/run21/20230924094328325_btl_326
diff tmp 0:00:02.423512
ini 2023-09-24 09:43:28.448489
in  rien
ini next 2023-09-24 09:43:30.872001
in  next 2023-09-24 09:43:31.128733
2023-09-24 10:03:58,954 - INFO - video is saved as ./demo/run21/20230924100357932_btl_568
diff tmp 0:00:01.189509
ini 2023-09-24 10:03:58.062336
in  rien
ini next 2023-09-24 10:03:59.251845
in  next 2023-09-24 10:03:59.445015
2023-09-24 10:18:55,822 - INFO - video is saved as ./demo/run21/2023092

### 0924 - 0925

In [13]:
IHM_date = '0924_25'
IHM_path = './00007_btl_new/demo_0922/Read_2023' + IHM_date + '.txt'

with open(IHM_path, 'r') as file:
    IHM_data = file.read()

# matches = re.findall(r'^(.*EtatVoyant\s*\|\s*(?:2|4).*)$', IHM_data, re.MULTILINE)
matches = re.findall(r'^.*EtatVoyant.*$', IHM_data, re.MULTILINE)

voyant_list = []
for match in matches:
    voyant_list.append(match.replace('\x00', ''))

pattern = r'(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}\.\d{3}) \+\d{2}:\d{2} \| \["Read"\] \| EtatVoyant \| (\d)'
matches = [re.search(pattern, data) for data in voyant_list]
dates = [match.group(1) for match in matches if match]
numbers = [match.group(2) for match in matches if match]

df_voyant_0924_25 = pd.DataFrame({
    'Date': dates,
    'EtatVoyant': numbers
})
df_voyant_0924_25['Date'] = pd.to_datetime(df_voyant_0924_25['Date'], format='%Y-%m-%d %H:%M:%S.%f')

In [14]:
log_file = './00007_btl_new/demo_0922/run21/log.txt'
attempts = find_close_attempts(log_file, df_voyant_0924_25)

log file has 11924 lines
2023-09-24 09:25:40,865 - INFO - video is saved as ./demo/run21/20230924092540206_btl_119
diff tmp 0:00:00.881836
ini 2023-09-24 09:25:40.326954
in  rien
ini next 2023-09-24 09:25:41.208790
in  next 2023-09-24 09:25:41.929096
2023-09-24 09:25:48,616 - INFO - video is saved as ./demo/run21/20230924092548369_btl_123
diff tmp 0:00:00.614386
ini 2023-09-24 09:25:48.158658
in  rien
ini next 2023-09-24 09:25:48.773044
in  next 2023-09-24 09:25:49.188931
2023-09-24 09:43:30,407 - INFO - video is saved as ./demo/run21/20230924094328325_btl_326
diff tmp 0:00:02.423512
ini 2023-09-24 09:43:28.448489
in  rien
ini next 2023-09-24 09:43:30.872001
in  next 2023-09-24 09:43:31.128733
2023-09-24 10:03:58,954 - INFO - video is saved as ./demo/run21/20230924100357932_btl_568
diff tmp 0:00:01.189509
ini 2023-09-24 10:03:58.062336
in  rien
ini next 2023-09-24 10:03:59.251845
in  next 2023-09-24 10:03:59.445015


In [15]:
log_file = './00007_btl_new/demo_0922/run22/log.txt'
attempts = find_close_attempts(log_file, df_voyant_0924_25)

log file has 25866 lines
2023-09-24 10:26:39,466 - INFO - video is saved as ./demo/run22/20230924102638703_btl_74
diff tmp 0:00:00.971928
ini 2023-09-24 10:26:38.841291
in  rien
ini next 2023-09-24 10:26:39.813219
in  next 2023-09-24 10:26:40.291743
2023-09-24 10:28:43,954 - INFO - video is saved as ./demo/run22/20230924102843268_btl_122
diff tmp 0:00:00.966454
ini 2023-09-24 10:28:43.403248
in  rien
ini next 2023-09-24 10:28:44.369702
in  next 2023-09-24 10:28:44.679308
2023-09-24 10:29:10,934 - INFO - video is saved as ./demo/run22/20230924102909927_btl_126
diff tmp 0:00:01.421461
ini 2023-09-24 10:29:10.053179
in  rien
ini next 2023-09-24 10:29:11.474640
in  next 2023-09-24 10:29:11.501541
2023-09-24 10:31:43,051 - INFO - video is saved as ./demo/run22/20230924103141920_btl_170
diff tmp 0:00:01.741298
ini 2023-09-24 10:31:42.049170
in  rien
ini next 2023-09-24 10:31:43.790468
in  next 2023-09-24 10:31:44.842504
2023-09-24 10:34:22,635 - INFO - video is saved as ./demo/run22/20230924

### 0927

In [21]:
IHM_date = '0927'
IHM_path = './00007_btl_new/demo_0927/Read_2023' + IHM_date + '.txt'

with open(IHM_path, 'r') as file:
    IHM_data = file.read()

# matches = re.findall(r'^(.*EtatVoyant\s*\|\s*(?:2|4).*)$', IHM_data, re.MULTILINE)
matches = re.findall(r'^.*EtatVoyant.*$', IHM_data, re.MULTILINE)

voyant_list = []
for match in matches:
    voyant_list.append(match.replace('\x00', ''))

pattern = r'(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}\.\d{3}) \+\d{2}:\d{2} \| \["Read"\] \| EtatVoyant \| (\d)'
matches = [re.search(pattern, data) for data in voyant_list]
dates = [match.group(1) for match in matches if match]
numbers = [match.group(2) for match in matches if match]

df_voyant_0927 = pd.DataFrame({
    'Date': dates,
    'EtatVoyant': numbers
})
df_voyant_0927['Date'] = pd.to_datetime(df_voyant_0927['Date'], format='%Y-%m-%d %H:%M:%S.%f')

In [32]:
log_file = './00007_btl_new/demo_0927/run4/log.txt'
attempts = find_close_attempts(log_file, df_voyant_0927)

log file has 1846 lines
2023-09-27 15:04:33,829 - INFO - TRICHE POTENTIELLE 2: ./demo/run4/20230927150433287_btl_89

diff tmp 0:00:00.366702
ini 2023-09-27 15:04:33.171760
out
in  rien
ini next 2023-09-27 15:04:33.538462
in  next 2023-09-27 15:04:33.623586
[1]


In [33]:
log_file = './00007_btl_new/demo_0927/run5/log.txt'
attempts = find_close_attempts(log_file, df_voyant_0927)

log file has 13835 lines
2023-09-27 15:37:02,106 - INFO - TRICHE POTENTIELLE 2: ./demo/run5/20230927153651646_btl_192

diff tmp 0:00:01.886340
ini 2023-09-27 15:36:50.002225
out
in  rien
ini next 2023-09-27 15:36:51.888565
in  next 2023-09-27 15:36:52.346574
[1]
2023-09-27 15:43:47,620 - INFO - video is saved as ./demo/run5/20230927154341260_btl_296

diff tmp 0:00:00.457808
ini 2023-09-27 15:43:40.803888
out
in  rien
ini next 2023-09-27 15:43:41.261696
in  next 2023-09-27 15:43:41.358330
[1]
2023-09-27 15:47:54,668 - INFO - TRICHE POTENTIELLE 2: ./demo/run5/20230927154752492_btl_342

diff tmp 0:00:01.412694
ini 2023-09-27 15:47:51.335752
out
in  rien
ini next 2023-09-27 15:47:52.748446
in  next 2023-09-27 15:47:52.890426
[1]
2023-09-27 15:49:43,090 - INFO - TRICHE POTENTIELLE 2: ./demo/run5/20230927154935408_btl_379

diff tmp 0:00:02.055002
ini 2023-09-27 15:49:33.599871
out
in  rien
ini next 2023-09-27 15:49:35.654873
in  next 2023-09-27 15:49:35.855060
[1]
2023-09-27 15:56:20,037 - I

In [34]:
log_file = './00007_btl_new/demo_0927/run6/log.txt'
attempts = find_close_attempts(log_file, df_voyant_0927)

log file has 2634 lines
2023-09-27 17:19:06,537 - INFO - TRICHE POTENTIELLE 2: ./demo/run6/20230927171900858_btl_164

diff tmp 0:00:02.835429
ini 2023-09-27 17:18:58.261879
out
in  rien
ini next 2023-09-27 17:19:01.097308
in  next 2023-09-27 17:19:01.644169
[1]
2023-09-27 17:20:37,321 - INFO - TRICHE POTENTIELLE 2: ./demo/run6/20230927172025461_btl_185

diff tmp 0:00:00.973715
ini 2023-09-27 17:20:24.759547
out
in  rien
ini next 2023-09-27 17:20:25.733262
in  next 2023-09-27 17:20:25.821625
[1]


In [35]:
log_file = './00007_btl_new/demo_0927/run7/log.txt'
attempts = find_close_attempts(log_file, df_voyant_0927)

log file has 13 lines


### 0929

In [39]:
IHM_date = '0929'
IHM_path = './00007_btl_new/demo_0927/Read_2023' + IHM_date + '.txt'

with open(IHM_path, 'r') as file:
    IHM_data = file.read()

# matches = re.findall(r'^(.*EtatVoyant\s*\|\s*(?:2|4).*)$', IHM_data, re.MULTILINE)
matches = re.findall(r'^.*EtatVoyant.*$', IHM_data, re.MULTILINE)

voyant_list = []
for match in matches:
    voyant_list.append(match.replace('\x00', ''))

pattern = r'(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}\.\d{3}) \+\d{2}:\d{2} \| \["Read"\] \| EtatVoyant \| (\d)'
matches = [re.search(pattern, data) for data in voyant_list]
dates = [match.group(1) for match in matches if match]
numbers = [match.group(2) for match in matches if match]

df_voyant_0929 = pd.DataFrame({
    'Date': dates,
    'EtatVoyant': numbers
})
df_voyant_0929['Date'] = pd.to_datetime(df_voyant_0929['Date'], format='%Y-%m-%d %H:%M:%S.%f')

In [40]:
log_file = './00007_btl_new/demo_0927/run11/log.txt'
attempts = find_close_attempts(log_file, df_voyant_0929)

log file has 78863 lines
2023-09-29 17:14:53,783 - INFO - TRICHE POTENTIELLE 2: ./demo/run11/20230929171449207_btl_67

diff tmp 0:00:02.623084
ini 2023-09-29 17:14:46.819172
out
in  rien
ini next 2023-09-29 17:14:49.442256
in  next 2023-09-29 17:14:51.521570
[1]
2023-09-29 17:16:28,635 - INFO - TRICHE POTENTIELLE 2: ./demo/run11/20230929171626752_btl_87

diff tmp 0:00:02.343154
ini 2023-09-29 17:16:24.654415
out
in  rien
ini next 2023-09-29 17:16:26.997569
in  next 2023-09-29 17:16:27.139985
[1]
2023-09-29 17:16:51,530 - INFO - TRICHE POTENTIELLE 2: ./demo/run11/20230929171650238_btl_91

diff tmp 0:00:02.116531
ini 2023-09-29 17:16:48.358442
out
in  rien
ini next 2023-09-29 17:16:50.474973
in  next 2023-09-29 17:16:50.736221
[1]
2023-09-29 17:17:46,345 - INFO - TRICHE POTENTIELLE 2: ./demo/run11/20230929171745132_btl_114

diff tmp 0:00:02.909911
ini 2023-09-29 17:17:42.486592
out
in  rien
ini next 2023-09-29 17:17:45.396503
in  next 2023-09-29 17:17:45.660972
[1]
2023-09-29 17:33:08,52

In [41]:
log_file = './00007_btl_new/demo_0927/run12/log.txt'
attempts = find_close_attempts(log_file, df_voyant_0929)

log file has 13 lines


In [42]:
log_file = './00007_btl_new/demo_0927/run13/log.txt'
attempts = find_close_attempts(log_file, df_voyant_0929)

log file has 41168 lines
2023-09-30 14:00:08,430 - INFO - video is saved as ./demo/run13/20230930140003781_btl_9

diff tmp 0:00:02.447086
ini 2023-09-30 14:00:01.588507
out
in  rien
ini next 2023-09-30 14:00:04.035593
in  next 2023-09-30 14:00:05.381524
[1]
2023-09-30 14:00:21,708 - INFO - video is saved as ./demo/run13/20230930140018604_btl_18

diff tmp 0:00:00.478897
ini 2023-09-30 14:00:18.376953
out
in  rien
ini next 2023-09-30 14:00:18.855850
in  next 2023-09-30 14:00:20.977342
[1]
2023-09-30 14:03:23,571 - INFO - video is saved as ./demo/run13/20230930140312946_btl_67

diff tmp 0:00:00.310543
ini 2023-09-30 14:03:12.886847
out
in  rien
ini next 2023-09-30 14:03:13.197390
in  next 2023-09-30 14:03:22.866700
[1]
2023-09-30 14:05:30,126 - INFO - video is saved as ./demo/run13/20230930140512260_btl_117

diff tmp 0:00:01.817213
ini 2023-09-30 14:05:10.716491
out
in  rien
ini next 2023-09-30 14:05:12.533704
in  next 2023-09-30 14:05:13.664942
[1]
2023-09-30 14:06:22,977 - INFO - TRICHE

2023-10-01 09:59:44,957 - INFO - TRICHE POTENTIELLE 2: ./demo/run13/20231001095943508_btl_1732

diff tmp 0:00:00.844626
ini 2023-10-01 09:59:42.441977
out
in  rien
ini next 2023-10-01 09:59:43.286603
in  next 2023-10-01 09:59:43.538307
[1]
2023-10-01 11:37:09,385 - INFO - TRICHE POTENTIELLE 2: ./demo/run13/20231001113636257_btl_2114

diff tmp 0:00:01.713081
ini 2023-10-01 11:36:34.790556
out
in  rien
ini next 2023-10-01 11:36:36.503637
in  next 2023-10-01 11:37:08.753114
[1]
2023-10-02 08:31:34,374 - INFO - video is saved as ./demo/run13/20231002083133403_btl_2157

diff tmp 0:00:00.713889
ini 2023-10-02 08:31:32.943720
out
in  rien
ini next 2023-10-02 08:31:33.657609
in  next 2023-10-02 08:31:33.878962
[1]
2023-10-02 08:32:33,698 - INFO - TRICHE POTENTIELLE 2: ./demo/run13/20231002083232414_btl_2170

diff tmp 0:00:00.387758
ini 2023-10-02 08:32:32.268979
out
in  rien
ini next 2023-10-02 08:32:32.656737
in  next 2023-10-02 08:32:33.068665
[1]
2023-10-02 08:34:57,844 - INFO - TRICHE POTE